In [1]:

from datasets import load_dataset

datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')


In [2]:

datasets


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [3]:

datasets["train"]


Dataset({
    features: ['text'],
    num_rows: 36718
})

In [4]:

datasets["train"][15:18]


{'text': [' The game takes place during the Second Europan War . Gallian Army Squad 422 , also known as " The Nameless " , are a penal military unit composed of criminals , foreign deserters , and military offenders whose real names are erased from the records and thereon officially referred to by numbers . Ordered by the Gallian military to perform the most dangerous missions that the Regular Army and Militia will not do , they are nevertheless up to the task , exemplified by their motto , Altaha Abilia , meaning " Always Ready . " The three main characters are No.7 Kurt Irving , an army officer falsely accused of treason who wishes to redeem himself ; Ace No.1 Imca , a female Darcsen heavy weapons specialist who seeks revenge against the Valkyria who destroyed her home ; and No.13 Riela Marcellis , a seemingly jinxed young woman who is unknowingly a descendant of the Valkyria . Together with their fellow squad members , these three are tasked to fight against a mysterious Imperial un

In [9]:

my_own_datasets = load_dataset("text", data_files={ "train": "/home/rcalix/Desktop/rc_train.txt", 
                                                    "validation": "/home/rcalix/Desktop/rc_validation.txt"} )


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [10]:

my_own_datasets


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 4
    })
})

In [12]:

my_own_datasets["train"][:]


{'text': ['The game takes place during the Second Europan War . Gallian Army Squad 422 , also known as " The Nameless " , are a penal military unit composed of criminals , foreign deserters , and military offenders whose real names are erased from the records and thereon officially referred to by numbers . Ordered by the Gallian military to perform the most dangerous missions that the Regular Army and Militia will not do , they are nevertheless up to the task , exemplified by their motto , Altaha Abilia , meaning " Always Ready . " The three main characters are No.7 Kurt Irving , an army officer falsely accused of treason who wishes to redeem himself ; Ace No.1 Imca , a female Darcsen heavy weapons specialist who seeks revenge against the Valkyria who destroyed her home ; and No.13 Riela Marcellis , a seemingly jinxed young woman who is unknowingly a descendant of the Valkyria . Together with their fellow squad members , these three are tasked to fight against a mysterious Imperial uni

In [13]:

my_own_datasets["validation"][:]


{'text': ['The gother other s , foreign deserters , and military offenders whose real names are erased from the records and thereon officially referred to by numbers . Ordered by the Gallian military to perform the most dangerous missions that the Regular Army and Militia will not do , they are nevertheless up to the task , exemplified by their motto , Altaha Abilia , meaning " Always Ready . " The three main characters are No.7 Kurt Irving , an army officer falsely accused of treason who wishes to redeem himself ; Ace No.1 Imca , a female Darcsen heavy weapons specialist who seeks revenge against the Valkyria who destroyed her home ; and No.13 Riela Marcellis , a seemingly jinxed young woman who is unknowingly a descendant of the Valkyria . Together with their fellow squad members , these three are tasked to fight against a mysterious Imperial unit known as Calamity Raven , consisting of mostly Darcsen soldiers .',
  "As tdfdfdfdfdfd  defer wise make Gallia lose face in the war . Whil

In [14]:

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


In [29]:



def show_random_elements(dataset, num_examples=20):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    
    picks = []
    
    for _ in range( num_examples ):
        
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame( dataset[picks] )        ## indexing 10 picks 
    
    print(df)
    print(dataset.features.items())
    
    for column, typ in dataset.features.items():
        print(column)
        print(typ)
        print(ClassLabel)
        ## The isinstance() function returns True if the specified object 
        ## is of the specified type, otherwise False
        if isinstance(typ, ClassLabel):
            print("Hello")
            df[column] = df[column].transform(lambda i: typ.names[i])
            print(typ.names[i])
            
    display(HTML(df.to_html()))


In [30]:

show_random_elements(datasets["train"])


                                                 text
0    The trailer advertising Mr. Burns ' search fo...
1                                                    
2       = = = Claims of flawed investigation = = = \n
3                                                    
4                                               2S \n
5    In his major theoretical statement , ' Opera ...
6    The Provincial Congress of New Jersey , made ...
7    The International Cricket Council ( ICC ) , r...
8                       = = Bishop of Hereford = = \n
9                          = = Interpretations = = \n
10   National Road Safety Council NGO , Armenia , ...
11                                                   
12   After writing ideas for over 70 songs , Towns...
13   Development of Dota 2 began in 2009 when IceF...
14   Villaret in Montagne , having successfully br...
15                                  = = Career = = \n
16                                   " Hollywood " \n
17   The last definite accou

,text
0,"The trailer advertising Mr. Burns ' search for an heir is a loose parody of the trailer for Toys , a 1992 comedy starring Robin Williams . \n"
1,
2,= = = Claims of flawed investigation = = = \n
3,
4,2S \n
5,"In his major theoretical statement , ' Opera and Drama ' ( 1852 ) , Wagner objected to the music of Meyerbeer , asserting its superficiality and incoherence in dramatic terms ; this work contains Wagner 's well @-@ known put @-@ down of Meyerbeer 's operas as ' effects without causes ' . It also contains the sardonic crack that ' [ Rossini ] never could have dreamt that it would some day occur to the Bankers , for whom he had always made their music , to make it for themselves ' . ' Jewishness in Music ' was reissued in 1869 , ( after Meyerbeer 's death ) in an extended form , with a far more explicit attack on Meyerbeer . This version was under Wagner 's own name – and as Wagner had by now a far greater reputation , his views obtained far wider publicity . These attacks on Meyerbeer ( which also included swipes at Felix Mendelssohn ) are regarded by Paul Lawrence Rose as a significant milestone in the growth of German anti @-@ Semitism . \n"
6,"The Provincial Congress of New Jersey , made up of elected delegates , formed in January 1776 to govern the colony . The Congress had Royal Governor William Franklin arrested on June 15 , declaring him "" an enemy to the liberties of this country "" . On July 2 , 1776 , New Jersey enacted the New Jersey State Constitution , soon after having empowered delegates to the Continental Congress , on June 21 , to join in a declaration of independence . The United States Declaration of Independence ended their colonial status . \n"
7,"The International Cricket Council ( ICC ) , responsible for running the game worldwide , conducted a trial in 2002 where lbw appeals could be referred to a match official , the third umpire , to review on television replays . The third umpire could only use technology to determine where the ball had pitched and if the batsman hit the ball with his bat . The ICC judged the experiment unsuccessful and did not pursue it . More trials followed in 2006 , although ball @-@ tracking technology remained unavailable to match officials . After a further series of trials , in 2009 the Umpire Decision Review System ( DRS ) was brought into international cricket where teams could refer the on @-@ field decisions of umpires to a third umpire who had access to television replays and technology such as ball tracking . According to the ICC 's general manager , Dave Richardson , DRS increased the frequency with which umpires awarded lbw decisions . In a 2012 interview , he said : "" Umpires may have realised that if they give someone out and DRS shows it was not out , then their decision can be rectified . So they might , I suppose , have the courage of their convictions a bit more and take a less conservative approach to giving the batsman out . I think if we 're totally honest , DRS has affected the game slightly more than we thought it would . "" \n"
8,= = Bishop of Hereford = = \n
9,= = Interpretations = = \n


In [31]:

model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"



For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text 

 The labels will be the same as the inputs, shifted to the left.


In [32]:

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)


In [33]:

def tokenize_function(examples):
    return tokenizer(examples["text"])


In [34]:

tokenized_datasets = datasets.map(
          tokenize_function, 
          batched=True, 
          num_proc=4, 
          remove_columns=["text"]
)


Parameter 'function'=<function tokenize_function at 0x2b3fb08abb80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [69]:

tokenized_datasets


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

In [70]:

## try several indices 

tokenized_datasets["train"][1]


{'input_ids': [238, 8576, 9441, 2987, 238, 252],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [71]:

# block_size = tokenizer.model_max_length
block_size = 128


In [72]:

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    
    ## concatenated_examples2 = {k: examples[k] for k in examples.keys()}
    ## print(concatenated_examples2)
    ## input()
    
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    
    # Split by chunks of max_len.
    result = {
        k: [ t[i : i + block_size] for i in range(0, total_length, block_size) ]
                    for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [74]:

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [75]:

tokenized_datasets.keys()


dict_keys(['test', 'train', 'validation'])

In [76]:

for k in tokenized_datasets.keys():
    print( tokenized_datasets[k] ) 


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4358
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 36718
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3760
})


In [77]:

examples1 =  tokenized_datasets


In [78]:

## concatenated_examples3 = {k: sum(examples1[k], []) for k in examples1.keys()}
## concatenated_examples3 
